In [1]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import ast

In [2]:
df = pd.read_pickle(r"Model_Train.pkl")

In [3]:
import ast

# Convert strings to lists (if needed)
df["nr_boil"] = df["nr_boil"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df["nr_pan"] = df["nr_pan"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to check if any polygon contains more than 1 panel
def has_multi_panel(nr_boil, nr_pan):
    return any(x > 1 for x in nr_boil) or any(x > 1 for x in nr_pan)

# Apply to DataFrame
df["multi_panel"] = df.apply(lambda row: has_multi_panel(row["nr_boil"], row["nr_pan"]), axis=1)

# Group and count
multi_panel_counts = df["multi_panel"].value_counts()

print("✅ Number of images with polygons containing >1 panel:")
print(multi_panel_counts)


✅ Number of images with polygons containing >1 panel:
multi_panel
False    1578
True      861
Name: count, dtype: int64


In [4]:
import os
import shutil
import pandas as pd

# Path to full dataset
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"

# Output folder for Labelme annotation
labelme_folder = r"C:\Users\gnvca\OneDrive\Desktop\JP\labelme_multi_panel"

# Ensure target folder exists
os.makedirs(labelme_folder, exist_ok=True)

# --- Filter only multi-panel images ---
def has_multi_panel(nr_boil, nr_pan):
    return any(x > 1 for x in nr_boil) or any(x > 1 for x in nr_pan)

# Ensure nr columns are lists (in case they come in as strings)
import ast
df["nr_boil"] = df["nr_boil"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df["nr_pan"] = df["nr_pan"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["multi_panel"] = df.apply(lambda row: has_multi_panel(row["nr_boil"], row["nr_pan"]), axis=1)
df_multi = df[df["multi_panel"] == True]

# --- Copy images (and existing annotations if available) ---
for img_id in df_multi["img_id"]:
    img_name = f"{img_id}.jpg"
    json_name = f"{img_id}.json"

    src_img = os.path.join(image_dir, img_name)
    dst_img = os.path.join(labelme_folder, img_name)

    if os.path.exists(src_img):
        shutil.copy2(src_img, dst_img)

    # Optional: also copy existing JSONs (if you've labeled some before)
    src_json = os.path.join(image_dir, json_name)
    if os.path.exists(src_json):
        shutil.copy2(src_json, os.path.join(labelme_folder, json_name))

print(f"✅ Copied {len(df_multi)} images to: {labelme_folder}")

# --- Launch labelme ---
os.system(f'labelme "{labelme_folder}"')


✅ Copied 861 images to: C:\Users\gnvca\OneDrive\Desktop\JP\labelme_multi_panel


1